# Importing

Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Data processing

Importing the datasets into a Dataframe

In [4]:
jester_items = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_items.csv')
jester_ratings = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_ratings.csv')
display(jester_items.head(3),jester_ratings.head(3))

,jokeId,jokeText
0,1,"A man visits the doctor. The doctor says ""I ha..."
1,2,This couple had an excellent relationship goin...
2,3,Q. What's 200 feet long and has 4 teeth? \n\nA...


,userId,jokeId,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281


As is shown above, only the 'jester_ratings' dataset is required for the creation of our RecSys model.

The jokeId will be renamed to itemId to have a more universal dataset.

In [5]:
jester_df = jester_ratings.rename(columns={'jokeId':'itemId'})